In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data=pd.read_csv(r"..\ChurnModelling using ANN\Churn_Modelling.csv")

In [57]:
X=data.iloc[:,3:13]
y=data.iloc[:,13]

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [59]:
geography=pd.get_dummies(X["Geography"], drop_first=True)
gender=pd.get_dummies(X["Gender"], drop_first=True)

In [60]:
X=pd.concat([X,geography, gender],axis=1)

In [61]:
X=X.drop(["Geography","Gender"], axis=1)

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=0)

In [63]:
from sklearn.preprocessing import StandardScaler

In [64]:
sc=StandardScaler()

In [65]:
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [96]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU, PReLU, ELU, Activation, Embedding, Flatten, BatchNormalization
from keras.activations import relu, sigmoid
from keras.layers import Dropout
from keras.initializers import HeUniform, GlorotUniform
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV


In [133]:
def create_model(layers, activation):
    model= Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1], activation=activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes, activation=activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1, kernel_initializer="GlorotUniform", activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [134]:
model=KerasClassifier(build_fn=create_model, verbose=0)

In [135]:
param_grid = {
    'model__layers': [[20], [40, 20], [45, 20, 10]], 
    'model__activation': ["sigmoid", "relu"],        
    'batch_size': [128, 256],
    'epochs': [30],
}


In [136]:
grid=GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

In [137]:
grid_result = grid.fit(X_train, y_train)

c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\

c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\tshah6\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\

In [139]:
print(grid_result.best_params_, grid_result.best_score_)

{'batch_size': 128, 'epochs': 30, 'model__activation': 'relu', 'model__layers': [40, 20]} 0.856625


In [142]:
y_pred=grid.predict(X_test)
y_pred=y_pred>0.5

In [143]:
from sklearn.metrics import accuracy_score
ac=accuracy_score(y_test, y_pred)
print(ac)

0.8625
